In [3]:
import cv2
import numpy as np
import sys

# ----- Config (tune as needed) -----
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 100, 200
MORPH_CLOSE_W_RATIO = 0.015
MIN_AR, MAX_AR = 2.0, 6.0
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.002, 0.2
RECTANGULARITY_MIN = 0.6
EDGE_DENSITY_MIN = 0.03
PLATE_W, PLATE_H = 240, 80

IMG_PATH = r"E:\Indian_Number_Plates\Sample_Images\Datacluster_number_plates (66).jpg"

def wait_or_quit():
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')):  # ESC or q
        cv2.destroyAllWindows()
        raise SystemExit

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(blur)
    return clahe

def connect_plate_bands(edges, width):
    se_w = max(3, int(width * MORPH_CLOSE_W_RATIO))
    kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (se_w, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel_h, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_h, iterations=1)
    return opened

def contour_rectangularity(cnt):
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    rect_area = max(1, w*h)
    return area / rect_area

def edge_density_in_rect(edges, rect):
    x,y,w,h = rect
    roi = edges[max(0,y):y+h, max(0,x):x+w]
    if roi.size == 0:
        return 0.0
    return float(np.count_nonzero(roi)) / float(roi.size)

def approx_plate_quad(cnt):
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(4,2)
    return None

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]   # tl
    rect[2] = pts[np.argmax(s)]   # br
    rect[1] = pts[np.argmin(diff)]# tr
    rect[3] = pts[np.argmax(diff)]# bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)
    return warped
#INITALIZATION COMPLETE, NEXT STEP

In [5]:
#NEXT STEP
bgr = cv2.imread(IMG_PATH)
if bgr is None:
    raise FileNotFoundError(f"Could not read: {IMG_PATH}")

cv2.imshow("1 - Original", bgr)
wait_or_quit()

SystemExit: 

D:\Newfolder\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
img = resize_keep_aspect(bgr, SHORT_SIDE_TARGET)
cv2.imshow("2 - Resized", img)
wait_or_quit()
#NEXT STEP
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("3 - Grayscale", gray)
wait_or_quit()
#NEXT STEP
gray_enh = enhance_gray(gray)
cv2.imshow("4 - Enhanced (Blur+CLAHE)", gray_enh)
wait_or_quit()
#NEXT STEP
edges = cv2.Canny(gray, CANNY_LOWER, CANNY_UPPER)
cv2.imshow("5 - Canny Edges", edges)
wait_or_quit()
#NEXT STEP
bands = connect_plate_bands(edges, img.shape[1])
cv2.imshow("6 - Morph Connected Bands", bands)
wait_or_quit()
#NEXT STEP


cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

vis_cnt = img.copy()
cv2.drawContours(vis_cnt, cnts, -1, (0,255,0), 2)
cv2.imshow("7 - All Contours", vis_cnt)
wait_or_quit()

h, w = img.shape[:2]
area_img = h * w
candidates = []
for c in cnts:
    x,y,ww,hh = cv2.boundingRect(c)
    ar = ww / float(hh) if hh > 0 else 1e9
    area_c = ww * hh
    if not (MIN_AREA_FRAC*area_img <= area_c <= MAX_AREA_FRAC*area_img):
        continue
    if not (MIN_AR <= ar <= MAX_AR):
        continue
    rectness = contour_rectangularity(c)
    if rectness < RECTANGULARITY_MIN:
        continue
    ed = edge_density_in_rect(edges, (x,y,ww,hh))
    if ed < EDGE_DENSITY_MIN:
        continue
    quad = approx_plate_quad(c)
    if quad is None:
        quad = np.array([[x,y],[x+ww,y],[x+ww,y+hh],[x,y+hh]], dtype=np.int32)
    candidates.append({"quad": quad, "bbox": (x,y,ww,hh), "score": float(rectness*ed)})

vis_cand = img.copy()
for cand in candidates:
    x,y,ww,hh = cand["bbox"]
    cv2.rectangle(vis_cand, (x,y), (x+ww,y+hh), (0,0,255), 2)
cv2.imshow("8 - Filtered Candidates", vis_cand)
wait_or_quit()

if candidates:
    candidates.sort(key=lambda d: d["score"], reverse=True)
    best = candidates[0]
    warped = warp_perspective(img, best["quad"], PLATE_W, PLATE_H)
else:
    warped = np.zeros((PLATE_H, PLATE_W, 3), dtype=np.uint8)
    cv2.putText(warped, "No candidate found", (10, PLATE_H//2),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

cv2.imshow("9 - Rectified Plate (Best)", warped)
wait_or_quit()
cv2.destroyAllWindows()


In [16]:
import cv2
import numpy as np
import sys

# ========= Config (tune as needed) =========
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 100, 200

# Morphology / sizing
HORIZ_SE_FRAC = 0.010                 # horizontal structuring element width fraction
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.0015, 0.35
ASPECT_MIN, ASPECT_MAX = 1.8, 6.5     # wider for motorcycle plates
RECTANGULARITY_MIN = 0.60             # allow rounded corners
EDGE_DENSITY_MIN = 0.035
WHITE_FRAC_MIN = 0.50
MIN_W_FRAC = 0.10                      # candidate must be >=10% of image width

PLATE_W, PLATE_H = 260, 120            # taller canonical crop suits bike plates

# Your image path
IMG_PATH = r"E:\Indian_Number_Plates\Sample_Images\Datacluster_number_plates (66).jpg"

# ========= Utilities =========
def wait_or_quit():
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')):  # ESC or q
        cv2.destroyAllWindows()
        sys.exit(0)

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    # Denoise
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    # Illumination correction: white top-hat to emphasize bright slab (plate)
    se_bg = cv2.getStructuringElement(cv2.MORPH_RECT, (41,41))  # tune 31–61
    tophat = cv2.morphologyEx(blur, cv2.MORPH_TOPHAT, se_bg)
    # Local contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(tophat)
    return clahe

def build_edge_map(img_bgr, gray_enh, slab_mask):
    # Vertical gradient emphasizes character strokes
    gx16 = cv2.Sobel(gray_enh, cv2.CV_16S, 1, 0, ksize=3)
    gx = cv2.convertScaleAbs(gx16)
    gx = cv2.normalize(gx, None, 0, 255, cv2.NORM_MINMAX)

    # Base edges
    canny = cv2.Canny(gray_enh, CANNY_LOWER, CANNY_UPPER)

    # Suppress orange/yellow lamps (common false positives)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_orange = np.array([10, 80, 80], dtype=np.uint8)
    upper_orange = np.array([35, 255, 255], dtype=np.uint8)
    lamp_mask = cv2.inRange(hsv, lower_orange, upper_orange)

    # Fuse: vertical edges on bright slab, minus lamp areas
    combo = cv2.bitwise_and(gx, gx, mask=slab_mask)
    edges = cv2.max(canny, combo)
    edges = cv2.bitwise_and(edges, edges, mask=cv2.bitwise_not(lamp_mask))
    # Strengthen with slab mask one more time
    edges = cv2.bitwise_and(edges, slab_mask)
    return edges, gx, lamp_mask

def connect_plate_bands(edges, width):
    w_h = max(5, int(width * HORIZ_SE_FRAC))
    ker_h = cv2.getStructuringElement(cv2.MORPH_RECT, (w_h, 3))
    ker_s = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, ker_h, iterations=1)
    closed = cv2.morphologyEx(closed, cv2.MORPH_CLOSE, ker_s, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, ker_s, iterations=1)
    return opened

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]    # tl
    rect[2] = pts[np.argmax(s)]    # br
    rect[1] = pts[np.argmin(diff)] # tr
    rect[3] = pts[np.argmax(diff)] # bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)

# ========= Main =========
def main(img_path):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print("Failed to read image")
        return

    # 1) Original
    cv2.imshow("1 - Original", bgr); wait_or_quit()

    # 2) Resize
    img = resize_keep_aspect(bgr, SHORT_SIDE_TARGET)
    cv2.imshow("2 - Resized", img); wait_or_quit()

    # 3) Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("3 - Grayscale", gray); wait_or_quit()

    # 4) Enhance (Top-hat + CLAHE)
    gray_enh = enhance_gray(gray)
    cv2.imshow("4a - Enhanced (TopHat+CLAHE)", gray_enh); wait_or_quit()

    # 4b) Adaptive slab mask: bright plate background
    slab = cv2.adaptiveThreshold(
        gray_enh, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 35, -5
    )
    cv2.imshow("4b - Slab Mask (Adaptive)", slab); wait_or_quit()

    # 5) Edge map with vertical emphasis and lamp suppression
    edges, gx, lamp_mask = build_edge_map(img, gray_enh, slab)
    cv2.imshow("5a - Vertical Grad (Sobel X)", gx); wait_or_quit()
    cv2.imshow("5b - Lamp Mask (HSV)", lamp_mask); wait_or_quit()
    cv2.imshow("5c - Fused Edges ∩ Slab", edges); wait_or_quit()

    # 6) Morphological band connection
    bands = connect_plate_bands(edges, img.shape[1])
    cv2.imshow("6 - Morph Connected Bands", bands); wait_or_quit()

    # 7) Contours (visualize all)
    cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    vis_cnt = img.copy()
    cv2.drawContours(vis_cnt, cnts, -1, (0,255,0), 2)
    cv2.imshow("7 - All Contours", vis_cnt); wait_or_quit()

    # 8) Candidate filtering (geometry + photometric + stroke tests)
    h, w = img.shape[:2]
    area_img = h * w
    MIN_W_PX = int(MIN_W_FRAC * w)

    candidates = []
    for c in cnts:
        # Convexity to avoid highly jagged shapes
        hull = cv2.convexHull(c)
        hull_area = cv2.contourArea(hull)
        cnt_area = cv2.contourArea(c)
        if hull_area < 1 or (cnt_area / hull_area) < 0.7:
            continue

        rect = cv2.minAreaRect(hull)
        (cx, cy), (rw, rh), angle = rect
        rw, rh = max(rw, 1), max(rh, 1)
        ar = rw / rh if rh > 0 else 1e9
        a_rect = rw * rh

        # Geometry
        if a_rect < MIN_AREA_FRAC*area_img or a_rect > MAX_AREA_FRAC*area_img:
            continue
        if not (ASPECT_MIN <= ar <= ASPECT_MAX):
            continue
        if rw < MIN_W_PX:
            continue

        # Rectangularity using hull vs minAreaRect
        rectness = cnt_area / a_rect
        if rectness < RECTANGULARITY_MIN:
            continue

        # Axis-aligned ROI for photometric tests
        x,y,bbw,bbh = cv2.boundingRect(hull)
        roi_edges = edges[y:y+bbh, x:x+bbw]
        roi_gray  = gray_enh[y:y+bbh, x:x+bbw]
        roi_slab  = slab[y:y+bbh, x:x+bbw]
        if roi_edges.size == 0:
            continue

        # Edge density
        edge_density = float(np.count_nonzero(roi_edges)) / float(roi_edges.size)
        if edge_density < EDGE_DENSITY_MIN:
            continue

        # Plate slab fraction (bright background dominance)
        slab_frac = np.count_nonzero(roi_slab) / float(roi_slab.size)
        if slab_frac < WHITE_FRAC_MIN:
            continue

        # Robust vertical-stroke measure
        _, roi_bin = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        vproj = (255 - roi_bin).sum(axis=0)
        thr = vproj.mean() + 0.35*vproj.std()
        strong = (vproj > thr).astype(np.uint8)
        stroke_cols = int(strong.sum())

        # Longest contiguous run of strong columns
        if stroke_cols == 0:
            continue
        split_idx = np.where(np.concatenate(([strong[0]],
                               strong[:-1] != strong[1:], [True])))[0]
        diffs = np.diff(split_idx)[::2]
        longest = int(diffs.max()) if diffs.size else 0

        if stroke_cols < max(12, int(0.05*bbw)):
            continue
        if longest < max(6, int(0.02*bbw)):
            continue

        # Quadrilateral from minAreaRect
        box = cv2.boxPoints(rect).astype(np.int32)

        score = (0.45*edge_density + 0.25*rectness +
                 0.15*slab_frac + 0.15*(stroke_cols/float(bbw)))
        candidates.append({"quad": box, "bbox": (x,y,bbw,bbh), "score": float(score)})

    # 9) Show filtered candidates
    vis_cand = img.copy()
    for cand in candidates:
        x,y,ww,hh = cand["bbox"]
        cv2.rectangle(vis_cand, (x,y), (x+ww,y+hh), (0,0,255), 2)
    cv2.imshow("8 - Filtered Candidates", vis_cand); wait_or_quit()

    # 10) Warp best candidate
    if candidates:
        candidates.sort(key=lambda d: d["score"], reverse=True)
        best = candidates[0]
        warped = warp_perspective(img, best["quad"], PLATE_W, PLATE_H)
        cv2.imshow("9 - Rectified Plate (Best)", warped); wait_or_quit()
    else:
        blank = np.zeros((PLATE_H, PLATE_W, 3), dtype=np.uint8)
        cv2.putText(blank, "No candidate found", (10, PLATE_H//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
        cv2.imshow("9 - Rectified Plate (Best)", blank); wait_or_quit()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(IMG_PATH)


In [21]:
import cv2
import numpy as np
import sys

# ========= Config (tune as needed) =========
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 80, 180      # slightly softer edges

# Morphology / sizing
HORIZ_SE_FRAC = 0.009                   # a bit narrower to avoid over-merge
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.001, 0.45
ASPECT_MIN, ASPECT_MAX = 1.5, 7.5       # wide for motorcycle variability
RECTANGULARITY_MIN = 0.50               # allow rounded corners more
EDGE_DENSITY_MIN = 0.02                 # lower to avoid misses
WHITE_FRAC_MIN = 0.45                   # allow dirt/shadow
MIN_W_FRAC = 0.08                       # plate can be smaller in frame

PLATE_W, PLATE_H = 280, 140             # taller canonical crop

# Your image path
IMG_PATH = r"E:\Indian_Number_Plates\Sample_Images\Datacluster_number_plates (66).jpg"

# ========= Utilities =========
def wait_or_quit():
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')):  # ESC or q
        cv2.destroyAllWindows()
        sys.exit(0)

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if h < w:
        new_h = short_target
        new_w = int(w * (short_target / h))
    else:
        new_w = short_target
        new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    se_bg = cv2.getStructuringElement(cv2.MORPH_RECT, (51,51))  # stronger top-hat
    tophat = cv2.morphologyEx(blur, cv2.MORPH_TOPHAT, se_bg)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8)).apply(tophat)
    return clahe

def build_slab(gray_enh):
    # Combine adaptive and Otsu for a robust bright-plate mask
    slab_adp = cv2.adaptiveThreshold(gray_enh, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, -5)
    _, slab_otsu = cv2.threshold(gray_enh, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    slab = cv2.bitwise_and(slab_adp, slab_otsu)
    # Clean slab: open then close
    ker = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    slab = cv2.morphologyEx(slab, cv2.MORPH_OPEN, ker, iterations=1)
    slab = cv2.morphologyEx(slab, cv2.MORPH_CLOSE, ker, iterations=2)
    return slab

def build_edge_map(img_bgr, gray_enh, slab_mask):
    gx16 = cv2.Sobel(gray_enh, cv2.CV_16S, 1, 0, ksize=3)
    gx = cv2.convertScaleAbs(gx16)
    gx = cv2.normalize(gx, None, 0, 255, cv2.NORM_MINMAX)
    canny = cv2.Canny(gray_enh, CANNY_LOWER, CANNY_UPPER)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_orange = np.array([10, 80, 80], dtype=np.uint8)
    upper_orange = np.array([35, 255, 255], dtype=np.uint8)
    lamp_mask = cv2.inRange(hsv, lower_orange, upper_orange)
    combo = cv2.bitwise_and(gx, gx, mask=slab_mask)
    edges = cv2.max(canny, combo)
    edges = cv2.bitwise_and(edges, edges, mask=cv2.bitwise_not(lamp_mask))
    edges = cv2.bitwise_and(edges, slab_mask)
    return edges

def connect_plate_bands(edges, width):
    w_h = max(5, int(width * HORIZ_SE_FRAC))
    ker_h = cv2.getStructuringElement(cv2.MORPH_RECT, (w_h, 3))
    ker_s = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, ker_h, iterations=1)
    closed = cv2.morphologyEx(closed, cv2.MORPH_CLOSE, ker_s, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, ker_s, iterations=1)
    return opened

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]    # tl
    rect[2] = pts[np.argmax(s)]    # br
    rect[1] = pts[np.argmin(diff)] # tr
    rect[3] = pts[np.argmax(diff)] # bl
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)

def propose_fallback_from_slab(img, slab):
    # Assume plate is lower half of image under the tail lamp; compute largest bright rectangle
    h, w = slab.shape
    lower = slab[int(0.45*h):, :]
    contours, _ = cv2.findContours(lower, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        contours, _ = cv2.findContours(slab, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        y_off = 0
    else:
        y_off = int(0.45*h)
    # Choose the largest reasonably wide rect
    best = None
    best_score = -1
    for c in contours:
        rect = cv2.minAreaRect(c)
        (cx, cy), (rw, rh), _ = rect
        rw, rh = max(rw,1), max(rh,1)
        ar = rw / rh if rh>0 else 1e9
        if rw < 0.08*w or not (1.3 <= ar <= 8.0):
            continue
        area = rw*rh
        if area > best_score:
            best_score = area
            box = cv2.boxPoints(rect).astype(np.int32)
            box[:,1] += y_off
            best = box
    return best

# ========= Main =========
def main(img_path):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print("Failed to read image")
        return

    # 1) Original
    cv2.imshow("1 - Original", bgr); wait_or_quit()

    # 2) Resize
    img = resize_keep_aspect(bgr, SHORT_SIDE_TARGET)
    cv2.imshow("2 - Resized", img); wait_or_quit()

    # 3) Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("3 - Grayscale", gray); wait_or_quit()

    # 4) Enhance (Top-hat + CLAHE)
    gray_enh = enhance_gray(gray)
    cv2.imshow("4 - Enhanced", gray_enh); wait_or_quit()

    # 4b) Plate slab mask
    slab = build_slab(gray_enh)
    cv2.imshow("4b - Slab Mask", slab); wait_or_quit()

    # 5) Fused edge map
    edges = build_edge_map(img, gray_enh, slab)
    cv2.imshow("5 - Fused Edges ∩ Slab", edges); wait_or_quit()

    # 6) Morphological band connection
    bands = connect_plate_bands(edges, img.shape[1])
    cv2.imshow("6 - Morph Connected Bands", bands); wait_or_quit()

    # 7) Contours (visualize all)
    cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    vis_cnt = img.copy()
    cv2.drawContours(vis_cnt, cnts, -1, (0,255,0), 2)
    cv2.imshow("7 - All Contours", vis_cnt); wait_or_quit()

    # 8) Candidate filtering (relaxed + re-ranked)
    h, w = img.shape[:2]
    area_img = h * w
    MIN_W_PX = int(MIN_W_FRAC * w)
    candidates = []

    for c in cnts:
        hull = cv2.convexHull(c)
        rect = cv2.minAreaRect(hull)
        (cx, cy), (rw, rh), angle = rect
        rw, rh = max(rw, 1), max(rh, 1)
        ar = rw / rh if rh > 0 else 1e9
        a_rect = rw * rh

        if a_rect < MIN_AREA_FRAC*area_img or a_rect > MAX_AREA_FRAC*area_img:
            continue
        if not (ASPECT_MIN <= ar <= ASPECT_MAX):
            continue
        if rw < MIN_W_PX:
            continue

        # Photometric tests in ROI
        x,y,bbw,bbh = cv2.boundingRect(hull)
        roi_edges = edges[y:y+bbh, x:x+bbw]
        roi_slab  = slab[y:y+bbh, x:x+bbw]
        roi_gray  = gray_enh[y:y+bbh, x:x+bbw]
        if roi_edges.size == 0: 
            continue

        edge_density = float(np.count_nonzero(roi_edges)) / float(roi_edges.size)
        if edge_density < EDGE_DENSITY_MIN:
            continue

        slab_frac = np.count_nonzero(roi_slab) / float(roi_slab.size)
        if slab_frac < WHITE_FRAC_MIN:
            continue

        # Stroke cues (robust)
        _, roi_bin = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        vproj = (255 - roi_bin).sum(axis=0)
        thr = vproj.mean() + 0.30*vproj.std()
        strong = (vproj > thr).astype(np.uint8)
        stroke_cols = int(strong.sum())
        if stroke_cols < max(8, int(0.04*bbw)):
            continue

        # Score
        rectness = cv2.contourArea(hull) / a_rect
        box = cv2.boxPoints(rect).astype(np.int32)
        score = (0.40*edge_density + 0.25*rectness +
                 0.20*slab_frac + 0.15*(stroke_cols/float(max(1,bbw))))
        candidates.append({"quad": box, "bbox": (x,y,bbw,bbh), "score": float(score)})

    # Fallback proposal if nothing survived
    if not candidates:
        fb = propose_fallback_from_slab(img, slab)
        if fb is not None:
            candidates.append({"quad": fb, "bbox": cv2.boundingRect(fb), "score": 0.0})

    # 9) Show filtered candidates
    vis_cand = img.copy()
    for cand in candidates:
        x,y,ww,hh = cand["bbox"]
        cv2.rectangle(vis_cand, (x,y), (x+ww,y+hh), (0,0,255), 2)
    cv2.imshow("8 - Filtered/Fallback Candidates", vis_cand); wait_or_quit()

    # 10) Warp best candidate
    if candidates:
        candidates.sort(key=lambda d: d["score"], reverse=True)
        best = candidates[0]
        warped = warp_perspective(img, best["quad"], PLATE_W, PLATE_H)
        cv2.imshow("9 - Rectified Plate (Best)", warped); wait_or_quit()
    else:
        blank = np.zeros((PLATE_H, PLATE_W, 3), dtype=np.uint8)
        cv2.putText(blank, "No candidate found", (10, PLATE_H//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
        cv2.imshow("9 - Rectified Plate (Best)", blank); wait_or_quit()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(IMG_PATH)


In [25]:
import cv2
import numpy as np
import sys

# ========= Config =========
SHORT_SIDE_TARGET = 720
CANNY_LOWER, CANNY_UPPER = 80, 180

LOWER_TRIM_FRAC = 0.15
UPPER_TRIM_FRAC = 0.25
LEFT_RIGHT_MARGIN = 0.05

HORIZ_SE_FRAC = 0.010
MIN_AREA_FRAC, MAX_AREA_FRAC = 0.001, 0.45
ASPECT_MIN, ASPECT_MAX = 1.6, 7.0
RECTANGULARITY_MIN = 0.50
EDGE_DENSITY_MIN = 0.02
WHITE_FRAC_MIN = 0.45
MIN_W_FRAC = 0.10               # bump back to 10% to avoid tiny patches

CHAR_H_MIN_FRAC = 0.35          # text component height fraction bounds
CHAR_H_MAX_FRAC = 0.90
CHAR_MIN_COUNT = 4              # at least 4 tall dark components

PLATE_W, PLATE_H = 280, 140

IMG_PATH = r"E:\Indian_Number_Plates\Sample_Images\Datacluster_number_plates (66).jpg"

# ========= Utils =========
def wait_or_quit():
    k = cv2.waitKey(0) & 0xFF
    if k in (27, ord('q')): 
        cv2.destroyAllWindows(); sys.exit(0)

def resize_keep_aspect(img, short_target):
    h, w = img.shape[:2]
    if h < w:
        new_h = short_target; new_w = int(w * (short_target / h))
    else:
        new_w = short_target; new_h = int(h * (short_target / w))
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

def enhance_gray(gray):
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    se_bg = cv2.getStructuringElement(cv2.MORPH_RECT, (51,51))
    tophat = cv2.morphologyEx(blur, cv2.MORPH_TOPHAT, se_bg)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8)).apply(tophat)
    return clahe

def build_slab(gray_enh):
    slab_adp = cv2.adaptiveThreshold(gray_enh, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, -5)
    _, slab_otsu = cv2.threshold(gray_enh, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    slab = cv2.bitwise_and(slab_adp, slab_otsu)
    ker = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    slab = cv2.morphologyEx(slab, cv2.MORPH_OPEN, ker, iterations=1)
    slab = cv2.morphologyEx(slab, cv2.MORPH_CLOSE, ker, iterations=2)
    return slab

def build_edge_map(img_bgr, gray_enh, slab_mask):
    gx16 = cv2.Sobel(gray_enh, cv2.CV_16S, 1, 0, ksize=3)
    gx = cv2.convertScaleAbs(gx16)
    gx = cv2.normalize(gx, None, 0, 255, cv2.NORM_MINMAX)
    canny = cv2.Canny(gray_enh, CANNY_LOWER, CANNY_UPPER)
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_orange = np.array([10, 80, 80], dtype=np.uint8)
    upper_orange = np.array([35, 255, 255], dtype=np.uint8)
    lamp_mask = cv2.inRange(hsv, lower_orange, upper_orange)
    combo = cv2.bitwise_and(gx, gx, mask=slab_mask)
    edges = cv2.max(canny, combo)
    edges = cv2.bitwise_and(edges, edges, mask=cv2.bitwise_not(lamp_mask))
    edges = cv2.bitwise_and(edges, slab_mask)
    return edges

def restrict_roi(img, mask):
    h, w = img.shape[:2]
    top = int(UPPER_TRIM_FRAC * h)
    bot = h - int(LOWER_TRIM_FRAC * h)
    left = int(LEFT_RIGHT_MARGIN * w)
    right = w - int(LEFT_RIGHT_MARGIN * w)
    roi_mask = np.zeros_like(mask)
    roi_mask[top:bot, left:right] = 255
    return cv2.bitwise_and(mask, roi_mask), (top, left, bot, right)

def connect_plate_bands(edges, width):
    w_h = max(5, int(width * HORIZ_SE_FRAC))
    ker_h = cv2.getStructuringElement(cv2.MORPH_RECT, (w_h, 3))
    ker_s = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, ker_h, iterations=1)
    closed = cv2.morphologyEx(closed, cv2.MORPH_CLOSE, ker_s, iterations=1)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, ker_s, iterations=1)
    return opened

def order_quad_points(pts):
    rect = np.zeros((4,2), dtype="float32")
    s = pts.sum(axis=1); diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]; rect[2] = pts[np.argmax(s)]
    rect[1] = pts[np.argmin(diff)]; rect[3] = pts[np.argmax(diff)]
    return rect

def warp_perspective(image, quad_pts, out_w=PLATE_W, out_h=PLATE_H):
    rect = order_quad_points(quad_pts.astype("float32"))
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (out_w, out_h), flags=cv2.INTER_CUBIC)

def merge_boxes_same_row(boxes, y_tol=0.12):
    # Merge boxes whose vertical centers are close (same row)
    if not boxes: return []
    centers = [((b[0]+b[0]+b[2]+b[2])//4, (b[1]+b[1]+b[3]+b[3])//4) for (b) in
               [(x,y,x+w,y+h) for (x,y,w,h) in boxes]]
    ys = [c[1] for c in centers]
    h_mean = np.mean([b[3] for b in boxes])
    merged = []
    used = [False]*len(boxes)
    for i in range(len(boxes)):
        if used[i]: continue
        x,y,w,h = boxes[i]; yc = ys[i]
        group = [i]; gx, gy, gX, gY = x, y, x+w, y+h
        used[i] = True
        for j in range(i+1, len(boxes)):
            if used[j]: continue
            x2,y2,w2,h2 = boxes[j]; yc2 = ys[j]
            if abs(yc2 - yc) <= y_tol * h_mean:
                used[j] = True
                group.append(j)
                gx, gy, gX, gY = min(gx,x2), min(gy,y2), max(gX,x2+w2), max(gY,y2+h2)
        merged.append((gx,gy,gX-gx,gY-gy))
    return merged

def count_dark_components(roi_gray, roi_bin):
    inv = 255 - roi_bin
    cnts, _ = cv2.findContours(inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h = roi_gray.shape[0]
    tall = 0
    for c in cnts:
        x,y,wc,hc = cv2.boundingRect(c)
        if CHAR_H_MIN_FRAC*h <= hc <= CHAR_H_MAX_FRAC*h and wc >= 2:
            tall += 1
    return tall

# ========= Main =========
def main(img_path):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print("Failed to read image"); return

    # 1) Preprocess
    img = resize_keep_aspect(bgr, SHORT_SIDE_TARGET)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_enh = enhance_gray(gray)
    slab = build_slab(gray_enh)

    # Restrict search to plausible vertical band
    slab_roi, (top, left, bot, right) = restrict_roi(img, slab)
    edges_full = build_edge_map(img, gray_enh, slab)
    edges_roi = np.zeros_like(edges_full)
    edges_roi[top:bot, left:right] = edges_full[top:bot, left:right]
    bands = connect_plate_bands(edges_roi, img.shape[1])

    # 2) Initial components
    cnts, _ = cv2.findContours(bands, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h, w = img.shape[:2]
    area_img = h*w
    MIN_W_PX = int(MIN_W_FRAC * w)

    base_boxes = []
    for c in cnts:
        x,y,bw,bh = cv2.boundingRect(c)
        if bw < MIN_W_PX: continue
        if bw*bh < MIN_AREA_FRAC*area_img or bw*bh > MAX_AREA_FRAC*area_img: continue
        ar = bw/float(bh)
        if not (ASPECT_MIN <= ar <= ASPECT_MAX): continue
        base_boxes.append((x,y,bw,bh))

    # 3) Merge adjacent boxes on same row
    merged_boxes = merge_boxes_same_row(base_boxes)

    # 4) Re-score merged boxes with topology + stroke tests
    candidates = []
    for (x,y,bw,bh) in merged_boxes:
        # Photometric checks inside merged box
        roi_gray = gray_enh[y:y+bh, x:x+bw]
        if roi_gray.size == 0: continue
        _, roi_bin = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        white_frac = np.count_nonzero(roi_bin) / float(roi_bin.size)
        if white_frac < WHITE_FRAC_MIN: continue

        # Dark component count (characters)
        tall_count = count_dark_components(roi_gray, roi_bin)
        if tall_count < CHAR_MIN_COUNT: continue

        # Edge density on edges_roi
        roi_edges = edges_roi[y:y+bh, x:x+bw]
        edge_density = float(np.count_nonzero(roi_edges)) / float(max(1, roi_edges.size))
        if edge_density < EDGE_DENSITY_MIN: continue

        # Build quad and score
        box = np.array([[x,y],[x+bw,y],[x+bw,y+bh],[x,y+bh]], dtype=np.int32)
        score = 0.45*edge_density + 0.30*white_frac + 0.25*(tall_count/float(max(CHAR_MIN_COUNT, 8)))
        candidates.append({"quad": box, "bbox": (x,y,bw,bh), "score": float(score)})

    # 5) Fallback: largest slab rectangle in ROI if needed
    if not candidates:
        cnts_slab, _ = cv2.findContours(slab_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts_slab:
            c = max(cnts_slab, key=cv2.contourArea)
            x,y,bw,bh = cv2.boundingRect(c)
            x += left; y += top
            box = np.array([[x,y],[x+bw,y],[x+bw,y+bh],[x,y+bh]], dtype=np.int32)
            candidates.append({"quad": box, "bbox": (x,y,bw,bh), "score": 0.05})

    # 6) Visualization and warp
    vis = img.copy()
    for cand in candidates:
        x,y,bw,bh = cand["bbox"]
        cv2.rectangle(vis, (x,y), (x+bw,y+bh), (0,0,255), 2)
    cv2.imshow("Candidates (merged+topology)", vis); wait_or_quit()

    if candidates:
        candidates.sort(key=lambda d: d["score"], reverse=True)
        best = candidates[0]
        warped = warp_perspective(img, best["quad"], PLATE_W, PLATE_H)
        cv2.imshow("Rectified Plate", warped); wait_or_quit()
    else:
        blank = np.zeros((PLATE_H, PLATE_W, 3), dtype=np.uint8)
        cv2.putText(blank, "No candidate", (10, PLATE_H//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.imshow("Rectified Plate", blank); wait_or_quit()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(IMG_PATH)
